<a href="https://colab.research.google.com/github/muditbac/nn-learn-notebooks/blob/main/Basic_GPT_Tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# @title Data for training our tokenizer

!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-03-18 07:29:13--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.02s   

2024-03-18 07:29:13 (60.2 MB/s) - ‘input.txt’ saved [1115394/1115394]



In this colab, we will learn about shakespere and through him we will learn about tokenization in LLMs. So let's build a quick bpe tokenizer first.

**Quick intro about BPE**: at each iteration it merges most frequent two chracters into a new characters and replaces all of it.

Sample Text: aababcbab
- find most frequent pair -> ab (3)
- replace ab with new char X -> aXXcbX
- no repeating elements can be found.

BPE for LLMs are done in bytes space and handle for unicode characters

In [3]:
# @title Imports
import math

In [4]:
# @title unicode and it's chracters
# @markdown Unicode is a universal character encoding standard that includes roughly 100,000 characters to represent characters of different languages. Unicode is unbounded because there's no reason to bound it.

sample_text = "यूनिकोड"

print(sample_text+'\n')

for x in sample_text:
  print(x, ord(x))

यूनिकोड

य 2351
ू 2370
न 2344
ि 2367
क 2325
ो 2379
ड 2337


In [5]:
# @title unicode converted into bytes
for x in sample_text.encode('utf-8'):
  print(x)

224
164
175
224
165
130
224
164
168
224
164
191
224
164
149
224
165
139
224
164
161


# Basic BPE
In this appraoch, we apply BPE directly to the complete corpus. After learning ~1000 tokens, we encode the corpus and visualize the tokeinization process

In [6]:
text = open('input.txt').read()

In [7]:
# bpe has two steps first find the most common pair and replace it
def get_stats(array):
  count = {}
  for x,y in zip(array[:-1], array[1:]):
    count[(x,y)] = count.get((x,y), 0)+1
  return count

s = get_stats("aababcbab".encode())
sorted(s.items(), key=lambda x: -x[1])

[((97, 98), 3), ((98, 97), 2), ((97, 97), 1), ((98, 99), 1), ((99, 98), 1)]

In [8]:
def merge(original_array, pair, replace_with):
  new_array = []
  i = 0
  while i<len(original_array):
    if i<len(original_array)-1 and (original_array[i], original_array[i+1]) == pair:
      new_array.append(replace_with)
      i = i+2
    else:
      new_array.append(original_array[i])
      i = i+1
  return new_array

merge([1,2,3,4,3,2,3], (2,3), 9)


[1, 9, 4, 3, 9]

In [ ]:
num_tokens = 16

class Tokenizer():

  def train(self, text):
    raise NotImplemented

  def encode(self, text):
    raise NotImplemented

  def decode(self, tokens):
    raise NotImplemented

class BPETokenizer(Tokenizer):

  def __init__(self, num_tokens):
    self.num_tokens = num_tokens

  def train(self, text):
    text = list(text.encode('utf-8'))
    initial_len = len(text)
    self.merge_history = merge_history = {}
    for i in range(256, 256+self.num_tokens):
      stats = get_stats(text)
      most_common_pair = max(stats, key=lambda x: stats[x])
      text = merge(text, most_common_pair, i)
      merge_history[most_common_pair] = i
      print(f"merged {most_common_pair}: {stats[most_common_pair]} and replaced with {i}")

    final_len = len(text)
    print(f"initial len {initial_len}, final len {final_len}, compressed by factor of {initial_len / final_len}")

  def encode(self, text):
    # iteratively keep merging with lowest index, until nothing else can be merged
    text = list(text.encode('utf-8'))

    while 1:
      stats = get_stats(text)
      pair_with_lowest_index = min(stats, key=lambda x: self.merge_history.get(x, math.inf))
      print(pair_with_lowest_index)
    pass

tk = BPETokenizer(num_tokens=5000)
tk.train(text)

In [24]:

def encode(self, text):
  # iteratively keep merging with lowest index, until nothing else can be merged
  text = list(text.encode('utf-8'))

  while 1:
    stats = get_stats(text)
    pair_with_lowest_index = min(stats, key=lambda x: self.merge_history.get(x, math.inf))
    if pair_with_lowest_index not in self.merge_history:
      break
    text = merge(text, pair_with_lowest_index, self.merge_history[pair_with_lowest_index])
  return text

def decode(self, tokens, debug=True):
  flattened_tokens_to_chars = {i: [i] for i in range(256)}
  token_to_merge = {self.merge_history[i]: i for i in self.merge_history}
  final_chars = []

  def recursively_fetch(tkn):
    if tkn not in flattened_tokens_to_chars:
      pair = x,y = token_to_merge[tkn]
      flattened_tokens_to_chars[tkn] = recursively_fetch(x) + recursively_fetch(y)
    return flattened_tokens_to_chars[tkn]

  for x in tokens:
    final_chars.extend(recursively_fetch(x))
    final_chars.append(124)
  return bytearray(final_chars).decode('utf-8', errors='ignore')

sample_text = "Lafeu, consoling the Countess on the death of her husband and departure of her son. (All's Well That Ends Well)"
sample_text = "Five tribunes to defend their vulgar wisdoms, Of their own choice: one's Junius Brutus,"
sample_text = """He--to give fear to use and liberty,
Which have for long run by the hideous law,
As mice by lions--hath pick'd out an act,
Under whose heavy sense your brother's life
Falls into forfeit: he arrests him on it;
And follows close the rigour of the statute,
"""
sample_text = "we want to break free"
print(encode(tk, sample_text))
print(decode(tk, encode(tk, sample_text)))

[538, 119, 906, 283, 2411, 413, 309, 101]
we |w|ant |to |break| f|re|e|


Sample Outputs
```
[70, 561, 3467, 3371, 4223, 1950, 118, 465, 1485, 288, 3802, 377, 702, 2075, 2446, 2835, 1281, 541, 274, 691, 74, 347, 3447, 66, 414, 116, 441, 44]
F|ive |tribun|es to |defend |their |v|ul|gar| w|isdom|s, |Of| their |own |cho|ice|: |on|e's |J|un|ius |B|ru|t|us|,|
```

```
[72, 101, 530, 283, 803, 895, 399, 801, 284, 2679, 1491, 1266, 3291, 619, 894, 32, 1330, 3283, 104, 440, 2428, 1157, 1530, 109, 1167, 471, 306, 274, 115, 530, 570, 367, 657, 384, 600, 2915, 504, 277, 1043, 580, 3146, 3336, 2734, 379, 332, 2012, 325, 622, 394, 70, 493, 2114, 283, 300, 2403, 316, 2364, 272, 449, 900, 1675, 960, 316, 589, 1751, 259, 1702, 379, 295, 335, 103, 323, 1828, 2686, 491, 452]
H|e|--|to |give |fear| to |use |and |liber|ty|,
Which| have |for |long| |run| by the |h|id|eous |law|,
As |m|ice |by |li|on|s|--|hath| p|ick|'d |out |an a|ct|,
|Un|der| whose |heavy |sen|se |your| brother|'s |lif|e
|F|all|s in|to |for|fe|it|: he |ar|res|ts |him |on |it|;
And |follow|s |clo|se |the |ri|g|our| of the |stat|ut|e,
|

```

The problem with above BPE
- it creates tokens across multiple words. For example in the above example tokens like `es to ` and `s in` are split across chracters, which might increase noise in out training data
- number and math symbols in the corpus will be treated as single token which is not a good representation. instead, single digit and symbol should be one token

In [12]:
all_tokens = encode(tk, text)

## what percentage of all the tokens are used in the final output?

BPE merge two tokens to creates a new tokens. I had imageined that this will lead to lot's of smallers tokens not being used because the smaller tokens will be merged to created larger tokens during the encoding process. The below cells calculates all the unique set of tokens and it's ratio after training.

In [13]:
len(set(all_tokens))

1035

In [14]:
len(set(all_tokens).difference(range(256)))

971

In [15]:
len(set(all_tokens).difference(range(256))) / (1000)

0.971

# BPE with Regex Split (GPT-2)
- TBD